## RAG와 LLM을 활용한 질의응답 실습
- LLM api 호출시 RAG를 통해 연관 정보 추출 후 prompt에 추가하는 실습
- RAG 추가 전후로 평가 진행

In [1]:
import pickle

with open('./data/res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

num_questions = 10

questions = rag_data['questions'][:num_questions]
contexts = rag_data['contexts'][:num_questions]
answers = rag_data['answers'][:num_questions]
len(questions)

10

## 평가
- RAGAS의 Anwer Correctness 평가
    - 모델의 답변과 실제 정답 간의 일치율 측정

## Baseline (w/o RAG)

In [2]:
from tqdm.notebook import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question .
    Question:
    {questions[i]}
    """ 
    
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness
import keyring
import os

os.environ['OPENAI_API_KEY'] = keyring.get_password('openai', 'key_for_windows')

data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

{'answer_correctness': 0.3123}

In [5]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [6]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 글로벌 경제 성장률이 둔화되면서 중앙은행들은 경제를 부양하기 위해 금리를 낮추는 정책을 채택했습니다. 저금리는 기업과 소비자들이 더 쉽게 자금을 빌릴 수 있게 하여 소비와 투자를 촉진하려는 목적이 있습니다.

2. **인플레이션 압력 감소**: 많은 선진국에서 인플레이션이 낮은 수준을 유지하고 있습니다. 인플레이션이 낮을 때 중앙은행들은 금리를 낮게 유지할 수 있는 여유가 생깁니다. 이는 경제를 자극하기 위한 또 다른 이유가 됩니다.

3. **중앙은행의 통화 정책**: 미국 연방준비제도(Fed), 유럽중앙은행(ECB), 일본은행(BOJ) 등 주요 중앙은행들이 양적 완화(QE)와 같은 비전통적인 통화 정책을 통해 시장에 유동성을 공급하고 금리를 낮추는 정책을 지속적으로 시행해왔습니다.

4. **글로벌 불확실성**: 무역 전쟁, 지정학적 리스크, 브렉시트 등 글로벌 불확실성이 증가하면서 안전자산에 대한 수요가 증가했습니다. 이는 국채와 같은 안전자산의 수요를 높여 금리를 낮추는 효과를 가져왔습니다.

5. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 저축률이 높아지고 소비가 줄어드는 경향이 있습니다. 이는 자본의 공급이 수요를 초과하게 만들어 금리를 낮추는 요인으로 작용합니다.

6. **기술 발전과 생산성 증가**: 기술 발전과 생산성 증가로 인해 기업들이 더 적은 자본으로 더 많은 생산을 할 수 있게 되면서 자본 수요가 줄어들고, 이는 금리 하락으로 이어질 수 있습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다. 중앙은행들은 이러한 환경에서 경제를 안정시키고 성장시키기 위해 다양한 정책을 모색하고 있습니다.


## LLM + RAG
- text-embedding-3-large 임베딩 모델 사용

In [9]:
import numpy as np 
from tqdm.notebook import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

def retrieve_context(question, contexts):
    qustion_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')
    
    similarities = [cosine_similarity(qustion_embedding, context_embedding) for context_embedding in context_embeddings]
    
    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.
    
    Context:
    {context}
    
    Question:
    {questions[i]}
    """
    
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, answer_relevancy, context_recall, context_precision

data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

scores = evaluate(dataset, metrics=[answer_correctness])
score

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

{'answer_correctness': 0.3123}

In [14]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행했습니다. 이로 인해 글로벌 저금리 현상이 부각되었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여했습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상이 발생했습니다.

3. **코로나 팬데믹**: 최근 코로나 사태도 저금리 현상을 지속시키는 요인 중 하나입니다. 팬데믹이 진정되더라도 글로벌 저금리 추세는 계속될 가능성이 높습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.


In [15]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
